In [ ]:
%load_ext autoreload
%autoreload 2
import os
from v4vapp_backend_v2.config.setup import InternalConfig
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())

CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()

In [ ]:
### This empties the server account and sends balances back to test account

from v4vapp_backend_v2.actions.hive_to_lightning import get_verified_hive_client
from nectar.account import Account
from nectar.amount import Amount
from v4vapp_backend_v2.hive.hive_extras import send_transfer
from pprint import pprint

hive_client, server_name = await get_verified_hive_client()
server_account = Account(server_name, blockchain_instance=hive_client)
pprint(server_account.balances.get("available", []))
for amount in server_account.balances.get("available", []):
    print(f"Server account {server_name} has {amount}")
    if amount.amount > 0:
        trx = await send_transfer(
            to_account="v4vapp-test",
            from_account=server_name,
            hive_client=hive_client,
            amount=amount,
            memo="Clearing balance transfer from v4vapp backend to v4vapp-test account",
        )
        pprint(f"Transfer transaction: {trx}")


In [ ]:
async with db_client as client:
    db = client.get_db()
    await db["hive_ops"].delete_many({})
    await db["ledger"].delete_many({})
